# CLI

> Command Line Interface

In [ ]:
#| default_exp cli

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
"""CLI for Obsidian to Quarto sync tool"""

from fastcore.script import *
from rich.prompt import Prompt, Confirm
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
import os
from pathlib import Path
import json
from typing import Dict, Optional, Tuple

import sys
import argparse

# Import your config functions and sync function
from markdown_sync.config import load_config, save_config, get_setting, set_setting
from markdown_sync.main import sync_obsidian_to_quarto  # Assuming you put the sync code in a separate module

In [ ]:
#| export

console = Console()

def validate_path(path_str: str, must_exist: bool = True, must_be_dir: bool = True) -> Path:
    """Validate and return Path object"""
    path = Path(path_str).resolve()
    
    if must_exist and not path.exists():
        raise ValueError(f"Path does not exist: {path}")
    
    if must_be_dir and path.exists() and not path.is_dir():
        raise ValueError(f"Path is not a directory: {path}")
        
    return path

def find_sync_pair_for_cwd() -> Optional[Tuple[str, str, str]]:
    """
    Check if current working directory matches any configured obsidian or quarto paths.
    Returns (obsidian_path, quarto_path, author) if found, None otherwise.
    """
    cwd = Path.cwd().resolve()
    config = load_config()
    
    for obsidian_path, settings in config.items():
        obsidian_path_obj = Path(obsidian_path).resolve()
        quarto_path_obj = Path(settings['quarto-path']).resolve()
        
        # Check if cwd is the obsidian folder or inside it
        try:
            cwd.relative_to(obsidian_path_obj)
            return obsidian_path, settings['quarto-path'], settings.get('author', 'Author')
        except ValueError:
            pass
            
        # Check if cwd is the quarto folder or inside it
        try:
            cwd.relative_to(quarto_path_obj)
            return obsidian_path, settings['quarto-path'], settings.get('author', 'Author')
        except ValueError:
            pass
    
    return None

def list_configured_pairs():
    """Display all configured sync pairs"""
    config = load_config()
    print(config)
    if not config:
        console.print("❌ No sync pairs configured. Use 'markdown_sync add' to add one.")
        return
    
    table = Table(title="Configured Sync Pairs", show_header=True, header_style="bold magenta")
    table.add_column("Obsidian Path", style="cyan")
    table.add_column("Quarto Path", style="green")
    table.add_column("Author", style="")
    table.add_column("Status", style="white")
    
    for obsidian_path, settings in config.items():
        if obsidian_path in ('quarto-path','obsidian-path','author'): continue
        
        try:
            quarto_path = settings['quarto-path']
        except Exception as e:
            continue
        
        author = settings.get('author', 'Not set')
        
        # Check if paths exist
        obs_exists = Path(obsidian_path).exists()
        qrt_exists = Path(quarto_path).exists()
        
        if obs_exists and qrt_exists:
            status = "✅ Both exist"
        elif obs_exists:
            status = "⚠️  Quarto missing"
        elif qrt_exists:
            status = "⚠️  Obsidian missing" 
        else:
            status = "❌ Both missing"
        
        cln_quarto =  ('/').join(quarto_path.split('/')[-2:]) + "/"
        cln_obsidian = ('/').join(obsidian_path.split('/')[-2:]) + "/"
        table.add_row(cln_obsidian, cln_quarto, author, status)
    
    console.print(table)


In [ ]:
#| export

@call_parse
def add(obsidian_path: str = None,  # Absolute path to Obsidian folder
        quarto_path: str = None,    # Absolute path to Quarto folder  
        author: str = None,         # Default author name for new files
        non_interactive: bool = False):  # Skip prompts
    """Add a new Obsidian to Quarto sync pair"""
    
    try:
        if not non_interactive:
            console.print(Panel("📁 Add New Sync Pair", style="bold blue"))
            console.print("[bold blue]Let's add a new Obsidian → Quarto sync configuration!")
            console.print()
        
        # Get obsidian path
        if not obsidian_path:
            if non_interactive:
                raise ValueError("❌ Non-interactive mode requires obsidian_path!")
            console.print("[bold ]📖 First, Which Obsidian folder would you like to sync?")
            console.print("[](Folders will be copied recursively, markdown files → quarto documents)")
            obsidian_path = Prompt.ask("\nPath to Obsidian folder")
        
        obsidian_path_obj = validate_path(obsidian_path, must_exist=True, must_be_dir=True)
        
        # Get quarto path
        if not quarto_path:
            if non_interactive:
                raise ValueError("❌ Non-interactive mode requires quarto_path!")
            console.print("[bold ]📝 Which Quarto folder should receive the synced files?\n")
            quarto_path = Prompt.ask("\nPath to Quarto folder")
        
        quarto_path_obj = validate_path(quarto_path, must_exist=False, must_be_dir=True)
        
        # Get author
        if not author:
            if non_interactive:
                author = "Author"  # Default for non-interactive
            else:
                console.print("[bold ]👤 What should be the default author name for new .qmd files?\n")
                author = Prompt.ask("\nDefault Author Name", default="Author")
        
        # Create quarto directory if it doesn't exist
        if not quarto_path_obj.exists():
            if non_interactive or Confirm.ask(f"📁 Create Quarto directory at {quarto_path_obj}?"):
                quarto_path_obj.mkdir(parents=True, exist_ok=True)
                console.print(f"✅ Created directory: {quarto_path_obj}")
            else:
                console.print("❌ Cancelled - Quarto directory required")
                return
        
        # Show summary
        if not non_interactive:
            cln_quarto =  ('/').join(str(quarto_path_obj).split('/')[-2:]) + "/"
            cln_obsidian = ('/').join(str(obsidian_path_obj).split('/')[-2:]) + "/"
            
            console.print("\n📋 Configuration Summary:")
            console.print(f"   Obsidian Path: {cln_obsidian}")
            console.print(f"   Quarto Path: {cln_quarto}")
            console.print(f"   Author Name: {author}")
            
            if not Confirm.ask("\n💾 Save this configuration?"):
                console.print("❌ Setup cancelled.")
                return
        
        # Save configuration
        config = load_config()
        config[str(obsidian_path_obj)] = {
            'quarto-path': str(quarto_path_obj), 
            'author': author
        }
        save_config(config)
        console.print("[bold]\n✅ Sync pair added successfully!")
        
    except Exception as e:
        console.print(f"❌ Error: {e}")


In [ ]:
#| export

@call_parse
def remove(obsidian_path: str = None,  # Obsidian path to remove
           list_first: bool = False):   # List options first
    """Remove a sync pair from configuration"""
    
    config = load_config()
    
    if not config:
        console.print("❌ No sync pairs configured.")
        return
    
    if list_first or not obsidian_path:
        console.print("📋 Current sync pairs:")
        list_configured_pairs()
        console.print()

    if not obsidian_path:
        console.print("[bold]Which sync pair would you like to remove?")
        paths = list(config.keys())
        for i, path in enumerate(paths, 1):
            cln_path = ('/').join(path.split('/')[-2:]) + "/"
            console.print(f"  {i}. {cln_path}")
        
        try:
            choice = Prompt.ask("Enter number or full path")
            if choice.isdigit():
                choice_idx = int(choice) - 1
                if 0 <= choice_idx < len(paths):
                    obsidian_path = paths[choice_idx]
                else:
                    console.print("❌ Invalid choice")
                    return
            else:
                obsidian_path = choice
        except KeyboardInterrupt:
            console.print("\n❌ Cancelled")
            return
    
    # Normalize the path
    obsidian_path = str(Path(obsidian_path).resolve())
    
    if obsidian_path not in config:
        console.print(f"❌ Sync pair not found: {obsidian_path}")
        return
    
    quarto_path = config[obsidian_path]['quarto-path']
    
    # Confirm removal
    if not Confirm.ask(f"🗑️  Remove sync pair?\n   {obsidian_path} → {quarto_path}"):
        console.print("❌ Cancelled")
        return
    
    del config[obsidian_path]
    save_config(config)
    
    console.print("✅ Sync pair removed!")


In [ ]:
#| export
@call_parse
def list_pairs():
    """List all configured sync pairs"""
    list_configured_pairs()


In [ ]:
#| export
def sync(obsidian_path: str = None,     # Specific obsidian path to sync
         dry_run: bool = False,         # Show what would be done without doing it
         all_pairs: bool = False):      # Sync all configured pairs
    """Sync Obsidian markdown files to Quarto"""
    
    config = load_config()
    
    if not config:
        console.print("❌ No sync pairs configured. Use 'markdown_sync add' to add one.")
        return
    
    pairs_to_sync = []
    
    if obsidian_path:
        # Sync specific path
        obsidian_path = str(Path(obsidian_path).resolve())
        if obsidian_path in config:
            settings = config[obsidian_path]
            pairs_to_sync.append((obsidian_path, settings['quarto-path'], settings.get('author', 'Author')))
        else:
            console.print(f"❌ Sync pair not found: {obsidian_path}")
            return
    else:
        # Try to detect based on current directory
        pair_info = find_sync_pair_for_cwd()

        if pair_info:   pairs_to_sync.append(pair_info)

        else:
            for obs_path, settings in config.items():
                pairs_to_sync.append((obs_path, settings['quarto-path'], settings.get('author', 'Author')))
    
    # Perform sync operations
    for obs_path, qrt_path, author in pairs_to_sync:
        cln_quarto =  ('/').join(qrt_path.split('/')[-2:]) + "/"
        cln_obsidian = ('/').join(obs_path.split('/')[-2:]) + "/"
        
        console.print(f"\n🔄 Syncing: [bold blue3]{cln_obsidian}")
        console.print(f"         → [bold blue3]{cln_quarto}")
        console.print("-" * 60)
        
        try:
            sync_obsidian_to_quarto(
                obsidian_root=obs_path,
                quarto_root=qrt_path, 
                dry_run=dry_run
            )
            console.print(f"✅ Sync completed for {obs_path}")
            
        except Exception as e:
            console.print(f"❌ Error syncing {obs_path}: {e}")
    
    if dry_run and pairs_to_sync:
        console.print(f"\n💡 This was a dry run. Use --no-dry-run to actually perform the sync.")


In [ ]:
#| export

def main():
    """Main entry point that dispatches to subcommands"""
    
    if len(sys.argv) == 1:
        # No arguments provided - show help
        console.print(Panel("📝 Markdown Sync", subtitle="Obsidian → Quarto", style="bold blue"))
        console.print("Available commands:")
        console.print("  add     - Add a new sync pair")
        console.print("  remove  - Remove a sync pair")  
        console.print("  list    - List configured sync pairs")
        console.print("  sync    - Perform sync operation")
        console.print()
        console.print("Examples:")
        console.print("  markdown_sync add")
        console.print("  markdown_sync sync --no-dry-run")
        console.print("  markdown_sync list")
        console.print()
        console.print("Use 'markdown_sync <command> --help' for command-specific help")
        return
    
    # Get the subcommand
    subcommand = sys.argv[1]
    
    # Remove the subcommand from sys.argv so the individual command parsers work correctly
    sys.argv = [sys.argv[0]] + sys.argv[2:]
    
    # Dispatch to the appropriate command
    if subcommand == "add":
        add()
    elif subcommand == "remove":
        remove() 
    elif subcommand == "list":
        list_pairs()
    elif subcommand == "sync":
        sync()
    else:
        console.print(f"❌ Unknown command: {subcommand}")
        console.print("Available commands: add, remove, list, sync")
        console.print("Use 'markdown_sync' with no arguments to see help")
        sys.exit(1)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()